In [1]:
import numpy as np
import pandas as pd
from http import HTTPStatus
from dashscope import Generation
from tqdm import trange

In [4]:
train=pd.read_csv('CLUTRR_train.csv')

<>:1: SyntaxWarning: invalid escape sequence '\I'
<>:1: SyntaxWarning: invalid escape sequence '\I'
C:\Users\12609\AppData\Local\Temp\ipykernel_23044\3364529929.py:1: SyntaxWarning: invalid escape sequence '\I'
  train=pd.read_csv('F:\\Files\\work\ICT\\PAPER_PREPREDUCIBILITY\\LLM_can_learn_RULES\\CLUTRR_train.csv')


In [5]:
train

,id,query,target,f_comb,edge_types,genders,task_split
0,7c062432-2915-422f-b84f-8aaeba9a50d3,"('Dionne', 'Kyle')",grandfather,father-father,"['father', 'father']","Dionne:female,Edward:male,Kyle:male,Benny:male",train
1,15a6777d-0e29-45c1-a7a7-46fd47946085,"('Robin', 'Shirley')",sister,daughter-aunt,"['daughter', 'aunt']","Robin:female,Serena:female,Shirley:female,Lisa...",train
2,524dad6a-cb41-447f-904b-8f9945819c48,"('Kyle', 'Patty')",granddaughter,son-daughter,"['son', 'daughter']","Kyle:male,Edward:male,Patty:female,Karen:female",train
3,76b1579c-2be2-4415-b204-3d9851c184c7,"('Kyle', 'Amanda')",granddaughter,daughter-daughter,"['daughter', 'daughter']","Kyle:male,Theresa:female,Amanda:female,Henry:male",train
4,412ffec6-9876-41f1-b4af-9b027e51453b,"('Shirley', 'Edward')",brother,son-uncle,"['son', 'uncle']","Shirley:female,James:male,Edward:male,Vernon:male",train
...,...,...,...,...,...,...,...
65649,d7f1be54-9dec-4150-986e-3350d7d1b15f,"('Allen', 'Eric')",father,brother-father,"['brother', 'father']","Allen:male,Carl:male,Eric:male,Michael:male",train
65650,d0281819-e4fb-4fa7-a417-3ace945850ef,"('Robin', 'Margaret')",niece,sister-daughter,"['sister', 'daughter']","Robin:female,Shirley:female,Margaret:female,Ar...",train
65651,f0ee8b3b-fd15-444d-98ee-056d0ea2b6b1,"('Michael', 'Eric')",father-in-law,wife-father,"['wife', 'father']","Michael:male,Marlene:female,Eric:male,Barbara:...",train
65652,f4e75f22-4f84-4ef2-ba51-df303e89eb70,"('Beverly', 'Harold')",father,son-aunt-sister-father,"['son', 'aunt', 'sister', 'father']","Beverly:female,Paul:male,Marie:female,Dorothy:...",train


In [55]:
train['head_tail_pairs'] = train['query'].apply(lambda x: [eval(x)[0],eval(x)[1]])

In [57]:
train['relation_path']=train['f_comb'].apply(lambda x: x.split('-'))

In [62]:
train.head()

,id,query,target,f_comb,edge_types,genders,task_split,head_tail_pairs,relation_path
0,7c062432-2915-422f-b84f-8aaeba9a50d3,"('Dionne', 'Kyle')",grandfather,father-father,"['father', 'father']","Dionne:female,Edward:male,Kyle:male,Benny:male",train,"[Dionne, Kyle]","[father, father]"
1,15a6777d-0e29-45c1-a7a7-46fd47946085,"('Robin', 'Shirley')",sister,daughter-aunt,"['daughter', 'aunt']","Robin:female,Serena:female,Shirley:female,Lisa...",train,"[Robin, Shirley]","[daughter, aunt]"
2,524dad6a-cb41-447f-904b-8f9945819c48,"('Kyle', 'Patty')",granddaughter,son-daughter,"['son', 'daughter']","Kyle:male,Edward:male,Patty:female,Karen:female",train,"[Kyle, Patty]","[son, daughter]"
3,76b1579c-2be2-4415-b204-3d9851c184c7,"('Kyle', 'Amanda')",granddaughter,daughter-daughter,"['daughter', 'daughter']","Kyle:male,Theresa:female,Amanda:female,Henry:male",train,"[Kyle, Amanda]","[daughter, daughter]"
4,412ffec6-9876-41f1-b4af-9b027e51453b,"('Shirley', 'Edward')",brother,son-uncle,"['son', 'uncle']","Shirley:female,James:male,Edward:male,Vernon:male",train,"[Shirley, Edward]","[son, uncle]"


In [82]:
def create_zero_shot_content(X):
    path_list=X['relation_path']
    relation=""
    for i in range(len(path_list)-1):
        relation+=path_list[i]+','
    relation+=path_list[-1]
    str="Context: The relations on the path from {"+X['head_tail_pairs'][0]+'} to {'+X['head_tail_pairs'][1]+'} are {'+relation+'}. Question: {'+X['head_tail_pairs'][0]+'} is {'+X['head_tail_pairs'][1]+'}’s what? \n\nAnswer: Let’s think step by step.'
    return str

train['zero_shot_content']=train.apply(create_zero_shot_content,axis=1)

In [94]:
train['length']=train['edge_types'].apply(lambda x: len(eval(x)))

In [170]:
datas=[]
for i in train.groupby('length'):
    i[1].reset_index(drop=True,inplace=True)
    datas.append(i[1].iloc[:2000,:])

for i in range(len(datas)):
    datas[i].to_csv(f"CLUTTER_test_{i}.csv", index=False)

## **zero_shot CoT**

In [184]:
model = "qwen-turbo"


def zero_shot_generation(content):
    responses = []
    messages = [{"role": "system", "content": ""}, {"role": "user", "content": content}]
    response = Generation.call(
        model=model,
        messages=messages,
        prompt=None,
        temperature=1.0,
        result_format="message",
    )
    if response.status_code == HTTPStatus.OK:
        responses.append(response.output.choices[0]["message"]["content"])
    else:
        print(
            "Request id: %s, Status code: %s, error code: %s, error message: %s"
            % (
                response.request_id,
                response.status_code,
                response.code,
                response.message,
            )
        )

    messages.append(
        {
            "role": response.output.choices[0]["message"]["role"],
            "content": response.output.choices[0]["message"]["content"],
        }
    )

    messages.append(
        {
            "role": "user",
            "content": "What's the final conclusion? Give me nothing but relation in one word.",
        }
    )
    response = Generation.call(
        model=model,
        messages=messages,
        prompt=None,
        temperature=1.0,
        result_format="message",
    )
    if response.status_code == HTTPStatus.OK:
        responses.append(response.output.choices[0]["message"]["content"])
    else:
        print(
            "Request id: %s, Status code: %s, error code: %s, error message: %s"
            % (
                response.request_id,
                response.status_code,
                response.code,
                response.message,
            )
        )

    return responses

In [185]:
zero_shots=[]
for i in range(len(datas)):
    zero_shot_result={'answer':[],'relation':[]}
    for j in trange(50):
        responsers=zero_shot_generation(datas[i]['zero_shot_content'][j])
        zero_shot_result['answer'].append(responsers[0])
        zero_shot_result['relation'].append(responsers[1].lower())
    temp=pd.DataFrame(zero_shot_result)
    temp['target']=datas[i]['target'][:50]
    zero_shots.append(temp)
zero_shots

100%|██████████| 50/50 [03:20<00:00,  4.02s/it]


[                                               answer            relation  \
 0   Given the context provided, we have a path fro...             father.   
 1   1. We are given that the relations between {Ro...               aunt.   
 2   Given the context provided, the relations betw...                son.   
 3   Given the context provided, we know that the r...             sister.   
 4   Given the context provided, we know that the r...              uncle.   
 5   1. We are given that the relations between {Ky...            brother.   
 6   Given the context provided, the relations betw...            sibling.   
 7   Given the context provided, we know that the r...           daughter.   
 8   Given the context provided, we know that the r...             husband   
 9   1. We are given that the relations between {Ve...                son.   
 10  Given the context provided, we know that the r...           daughter.   
 11  Given the context provided, we know that the r...          

In [186]:
for i in range(0,3):
    zero_shots[i].to_csv(f"zero_shot_{i}.csv", index=False)

## **5-shot CoT**

In [187]:
model="qwen-turbo"

def five_shots_generation(content):
    responses=[]
    content="Context: The relations on the path from Alan to Anthony are daughter, uncle, son. \n\n Question: Anthony is Alan’s what?\n\n Answer: For daughter’s uncle, we have daughter’s uncle is brother. So the relations are reduced to brother, son. For brother’s son, we have brother’s son is nephew. So the relations are reduced to nephew. Therefore, the answer is nephew. \n\nContext: The relations on the path from Annie to Carlos are brother, mother, son.\n\n Question: Carlos is Annie’s what? \n\n Answer: For brother’s mother, we have brother’s mother is mother. So the relations are reduced to mother, son. For mother’s son, we have mother’s son is brother. So the relations are reduced to brother. Therefore, the answer is brother. \n\nContext: The relations on the path from Beverly to Michelle are father, daughter, aunt. \n\nQuestion: Michelle is Beverly’s what? \n\nAnswer: For father’s daughter, we have father’s daughter is sister. So the relations are reduced to sister, aunt. For sister’s aunt, we have sister’s aunt is aunt. So the relations are reduced to aunt. Therefore, the answer is aunt. \n\nContext: The relations on the path from Lee to Jeanna are father, daughter, sister. \n\n Question: Jeanna is Lee’s what? \n\n Answer: For father’s daughter, we have father’s daughter is sister. So the relations are reduced to sister, sister. For sister’s sister, we have sister’s sister is sister. So the relations are reduced to sister. Therefore, the answer is sister. \n\n Context: The relations on the path from Craig to Molly are sister, father, mother. \n\nQuestion: Molly is Craig’s what? \n\nAnswer:For sister’s father, we have sister’s father is father. So the relations are reduced to father, mother. For father’s mother, we have father’s mother is grandmother. So the relations are reduced to grandmother. Therefore, the answer is grandmother.\n\n\n"+content
    messages = [{'role': 'system', 'content':""},
                {'role': 'user', 'content': content}]
    response = Generation.call(model=model,
                               messages=messages,
                               prompt=None,
                               temperature=1.0,
                               result_format="message")
    if response.status_code == HTTPStatus.OK:
        responses.append(response.output.choices[0]['message']['content'])
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
    
    messages.append({'role': response.output.choices[0]['message']['role'],
                         'content': response.output.choices[0]['message']['content']})
    
    messages.append({'role': 'user', 'content': "What's the final conclusion? Give me nothing but relation in one word."})
    response = Generation.call(model=model,
                               messages=messages,
                               prompt=None,
                               temperature=1.0,
                               result_format="message")
    if response.status_code == HTTPStatus.OK:
        responses.append(response.output.choices[0]['message']['content'])
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
    
    return responses

In [188]:
five_shots=[]
for i in range(len(datas)):
    five_shot_result={'answer':[],'relation':[]}
    for j in trange(50):
        responsers=five_shots_generation(datas[i]['zero_shot_content'][j])
        five_shot_result['answer'].append(responsers[0])
        five_shot_result['relation'].append(responsers[1].lower())
    temp=pd.DataFrame(five_shot_result)
    temp['target']=datas[i]['target'][:50]
    five_shots.append(temp)
five_shots

100%|██████████| 50/50 [02:59<00:00,  3.60s/it]


[                                               answer  \
 0   The context states that the relations between ...   
 1   Given the relations on the path from Robin to ...   
 2   For son's daughter, we have son's daughter is ...   
 3   Since the relations on the path from Kyle to A...   
 4   The relation given is that Shirley is Edward's...   
 5   From the given context, we know that the relat...   
 6   For the path given, Eddie is Karen's mother. S...   
 7   Since the path from Robin to Henry includes "d...   
 8   Since the relations on the path from Shirley t...   
 9   Since the relations on the path from Vernon to...   
 10  Given the context that Kyle and Kristen have t...   
 11  Since the relations on the path from Edward to...   
 12  For Dionne's father, we have the relation that...   
 13  Since Karen is Serena's husband's grandmother,...   
 14  Given the context, Kristen is Karen's husband....   
 15  Since the relation between Patty and Dionne is...   
 16  Since the

In [189]:
for j in range(0,3):
    five_shots[j].to_csv(f"five_shot_{j}.csv", index=False)

## **5-shot LtM**

In [190]:
five_s_L=datas[0]

In [191]:
model="qwen-turbo"

def five_shot_LTM(content):
    responses=[]
    content='Question: What is daughter’s uncle? \n\n Answer: Daughter’s uncle is mother. Question: What is brother’s mother? \n\n Answer: Brother’s mother is mother. \n\n Question: What is sister’s aunt? \n\n Answer: Sister’s aunt is aunt. \n\n Question: What is father’s daughter? \n\n Answer: Father’s daughter is sister. \n\n Question: What is sister’s father? \n\n Answer: Sister’s father is father.'+'What is {'+ content[0]+'} ’s {'+ content[1] +'}?'
    messages = [{'role': 'system', 'content':""},
                {'role': 'user', 'content': content}]
    response = Generation.call(model=model,
                               messages=messages,
                               prompt=None,
                               temperature=1.0,
                               result_format="message")
    if response.status_code == HTTPStatus.OK:
        responses.append(response.output.choices[0]['message']['content'])
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
    
    messages.append({'role': response.output.choices[0]['message']['role'],
                         'content': response.output.choices[0]['message']['content']})
    
    messages.append({'role': 'user', 'content': "What's the final conclusion? Give me nothing but relation in one word."})
    response = Generation.call(model=model,
                               messages=messages,
                               prompt=None,
                               temperature=1.0,
                               result_format="message")
    if response.status_code == HTTPStatus.OK:
        responses.append(response.output.choices[0]['message']['content'])
    else:
        print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
            response.request_id, response.status_code,
            response.code, response.message
        ))
    
    return responses

In [193]:
five_shot_LTM_result={'answer':[],'relation':[]}
for j in trange(50):
    responsers=five_shot_LTM(five_s_L['relation_path'][j])
    five_shot_LTM_result['answer'].append(responsers[0])
    five_shot_LTM_result['relation'].append(responsers[1].lower())
five_shot_LTM=pd.DataFrame(five_shot_LTM_result)
five_shot_LTM['target']=five_s_L['target'][:50]
five_shot_LTM.to_csv('five_shot_LTM.csv',index=False)
five_shot_LTM

100%|██████████| 50/50 [01:11<00:00,  1.42s/it]


,answer,relation,target
0,Answer: {Father}'s {father} is grandfather.,genealogy.,grandfather
1,Answer: {Daughter}'s {aunt} is aunt.,family.,sister
2,{Son}'s {daughter} is niece.,grandchild,granddaughter
3,{Daughter}'s {daughter} is granddaughter.,granddaughter,granddaughter
4,If we follow the pattern of the previous quest...,relative.,brother
5,{Granddaughter}'s {brother} is nephew.,relative.,grandson
6,"If ""mother"" refers to the person being asked a...",grandparent.,grandmother
7,"Based on the given answers, it seems like ther...",if we consider the provided answers consistent...,brother
8,{Husband}'s {daughter} is stepdaughter or chil...,family.,daughter
9,{Son}'s {grandmother} is grandmother.,family.,mother
